import libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Xây dựng - Nông nghiệp - Môi trường

find elements

In [3]:
# Function to crawl data from a page
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Get author and time information
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Get question title
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Get situation/summary
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Get detail link
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Send HTTP request to get detail page content
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Get answer from detail page
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Add article information to the list
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data

execute function

In [4]:
articles_data = []
for page_num in range(1, 400):
    page_url = f"https://chinhsachonline.chinhphu.vn/xay-dung-nong-nghiep-moi-truong/t1/13/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))

export to CSV

In [5]:
# Create DataFrame from the data list
df_bh = pd.DataFrame(articles_data)
df_bh.head(5)

,Name,Time,Question,Situation,Answer
0,Nguyễn Văn Hòa,11:05 30/07/2025,"Giấy chứng nhận khoanh vùng đất ở, tặng cho mộ...",Bố mẹ tôi đứng tên trên Giấy chứng nhận quyền ...,"Cục Quản lý đất đai, Bộ Nông nghiệp và Môi trư..."
1,Công ty CP Phát triển nhà C.T.,14:05 29/07/2025,"Quy trình giao đất, cho thuê đất thực hiện dự ...",Chúng tôi là chủ đầu tư dự án khu tái định cư ...,"Cục Quản lý Đất đai, Bộ Nông nghiệp và Môi trư..."
2,Hoàng Mạnh Đạt,07:05 28/07/2025,"Bị mất Giấy chứng nhận quyền sử dụng đất, xin ...",Đối với thủ tục cấp lại Giấy chứng nhận quyền ...,Bộ Nông nghiệp và Môi trường trả lời vấn đề nà...
3,Nguyễn Hồng,13:32 26/07/2025,Chuyển đất trồng cây thành đất nông nghiệp khá...,"Tôi xin hỏi, trường hợp chuyển mục đích sử dụn...","Cục Quản lý đất đai, Bộ Nông nghiệp và Môi trư..."
4,Ngô Ngọc Tân,11:05 25/07/2025,"Ghi nhận các đề xuất sửa quy định về quản lý, ...","Trong giai đoạn 1990-2010, thực hiện chủ trươn...","Cục Lâm nghiệp và Kiểm lâm, Bộ Nông nghiệp và ..."


In [6]:
 # Save DataFrame to CSV file
csv_file = '../source_fine_tuning/data/xd_nn_mt_data.csv'
df_bh.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../source_fine_tuning/data/xd_nn_mt_data.csv


In [7]:
df_bh=pd.read_csv("../source_fine_tuning/data/xd_nn_mt_data.csv",encoding='utf-8')

In [8]:
df_bh.shape

(2934, 6)